In [86]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

im2 = cv.imread("/home/sbsch/EC602/ChessBoard(3).png")
#Initial Build of Contours to check against

target_squares = [[1,0,"Pawn"],[0,1,"Knight"],[0,2,"Bishop"],[0,0,"Rook"],[0,3,"Queen"],[0,4,"King"]]
control_contours = []
# You can add an offset to the contours to shift them into the appropriate position of the board
for square in target_squares:
    i = square[1]
    j = square[0]
    piece = square[2]
    
    x = 123*j
    y = 123*i

    square = im2[x:x+123 , y:y+123]

    # Convert the image to grayscaleq
    gray = cv.cvtColor(square, cv.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce image noise and improve contour detection
    blurred = cv.GaussianBlur(gray, (3, 3), 0)

    # Use adaptive thresholding to create a binary image
    thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 7, 1)

    # Find contours in the thresholded image
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    temp_contours = []

    min_contour_area = 2000  # Minimum contour area to be considered a piece
    max_contour_area = 10000

    for cnt in contours:
        if min_contour_area <  cv.contourArea(cnt) < max_contour_area:
            temp_contours.append(cnt)
            control_contours.append(cnt)
            #np.save(f'{piece}.npy', temp_contours)
            
        else:
            pass
            
    

In [ ]:
### Testing the Control Contours

im2 = cv.imread("/home/sbsch/EC602/ChessBoard(3).png")
loaded_contours = np.load('Bishop.npy', allow_pickle=True)

cv.drawContours(im2, loaded_contours, -1, (0, 255, 0), 3)
cv.imshow("Detected Pieces", im2)
cv.waitKey(0)
cv.destroyAllWindows()

In [94]:
import cv2 as cv
from PIL import Image, ImageGrab 
import numpy as np
import matplotlib.pyplot as plt

letters = ['a','b','c','d','e','f','g','h']

#cv.imshow("ChessBoard", im2)

# pawn_contour = np.load('Pawn.npy', allow_pickle=True)
# knight_contour = np.load('Knight.npy', allow_pickle=True)
# bishop_contour = np.load('Bishop.npy', allow_pickle=True)
# rook_contour = np.load('Rook.npy', allow_pickle=True)
# queen_contour = np.load('Queen.npy', allow_pickle=True)
# king_contour = np.load('King.npy', allow_pickle=True)
#pieces_contours = [pawn_contour, knight_contour, bishop_contour, rook_contour, queen_contour, king_contour]



def shift_contour(contour, x_offset, y_offset):
    # Create a copy of the contour to avoid modifying the original
    shifted_contour = contour.copy()
    # Add the offset to all points in the contour
    shifted_contour[:,:,1] += x_offset  # Shift X coordinates
    shifted_contour[:,:,0] += y_offset  # Shift Y coordinates
    return shifted_contour

def piece_identifier(contour):
    similarity_array = []
    for piece in control_contours:
        similarity = cv.matchShapes(piece, contour, cv.CONTOURS_MATCH_I1, 0.0)
        similarity_array.append(similarity)
        
    if similarity_array.index(min(similarity_array)) == 0:
        return "p" #"Pawn"
    elif similarity_array.index(min(similarity_array)) == 1:
        return "n" #"Knight"
    elif similarity_array.index(min(similarity_array)) == 2:
        return "b" #"Bishop"
    elif similarity_array.index(min(similarity_array)) == 3:
        return "r" #"Rook"
    elif similarity_array.index(min(similarity_array)) == 4:
        return "q" #"Queen"
    elif similarity_array.index(min(similarity_array)) == 5:
        return "k" #"King"
    else:
        pass

def board_state(board_img_file):
    
    im2 = cv.imread(board_img_file)    
    black_pieces_contours = [ ]
    white_pieces_contours = [ ]
    board = [["." for _ in range(8)] for _ in range(8)]
    
    for i in range(8):
        for j in range(8):

            # You can add an offset to the contours to shift them into the appropriate position of the board
            x = 123*i
            y = 123*j

            square = im2[x:x+123 , y:y+123]

            # Convert the image to grayscaleq
            gray = cv.cvtColor(square, cv.COLOR_BGR2GRAY)

            # Apply GaussianBlur to reduce image noise and improve contour detection
            blurred = cv.GaussianBlur(gray, (3, 3), 0)

            # Use adaptive thresholding to create a binary image
            thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 7, 1)

            # Find contours in the thresholded image
            contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            
            temp_contours = []
            
            min_contour_area = 2000  # Minimum contour area to be considered a piece
            max_contour_area = 10000
            
            for cnt in contours:
                if min_contour_area <  cv.contourArea(cnt) < max_contour_area:
                    temp_contours.append(cnt)
                    piece_color = square[100, 64]
                    piece_type = piece_identifier(temp_contours[0])
                    #print(piece_color)
                else:
                    pass
                
            if len(temp_contours) > 0:
                if piece_color[0] == 87:
                    shifted_contour = shift_contour(temp_contours[0], x, y)
                    black_pieces_contours.append(shifted_contour)
                    #print(f"Piece: Black {piece_type} | Square: {letters[j]}{8-i}")
                    board[i][j] = f"{piece_type}"
                elif piece_color[0] == 249:
                    shifted_contour = shift_contour(temp_contours[0], x, y)
                    white_pieces_contours.append(shifted_contour)
                    #print(f"Piece: White {piece_type} | Square: {letters[j]}{8-i}")
                    board[i][j] = f"{piece_type.capitalize()}"
                else:
                    pass
            else:
                board[i][j] = "."

    for row in board:
        print(row)
        print()
    # Draw contours on the original image
    # im2_with_contours = im2.copy()
    # cv.drawContours(im2_with_contours, black_pieces_contours, -1, (255, 0, 0), 2)
    # cv.drawContours(im2_with_contours, white_pieces_contours, -1, (0, 255, 0), 2)

    # # Display the image with detected pieces
    # cv.imshow("Detected Pieces", im2_with_contours)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return board


In [ ]:
board1 = board_state("/home/sbsch/EC602/ChessBoard(5).png")
print("\n")
board2 = board_state("/home/sbsch/EC602/ChessBoard(6).png")

for i in range(8):
    for j in range(8):
        if board1[i][j] != board2[i][j]:
            print(f"Change at {letters[j]}{8-i} | {board1[i][j]} -> {board2[i][j]}")
            
            if board2[i][j] == ".":
                start_square = f"{letters[j]}{8-i}"
            else:
                end_square = f"{letters[j]}{8-i}"
        else:
            pass
        
print(f"{start_square}{end_square}")

['r', 'n', '.', 'k', '.', '.', '.', 'r']

['p', 'p', 'p', 'b', 'n', 'p', 'p', '.']

['.', '.', '.', '.', '.', '.', '.', '.']

['.', '.', '.', 'q', 'P', '.', '.', 'p']

['.', '.', 'P', 'N', '.', '.', '.', '.']

['.', '.', '.', '.', '.', 'P', '.', '.']

['P', 'P', '.', '.', 'B', '.', 'P', 'P']

['.', '.', 'R', 'Q', '.', 'R', 'K', '.']



['r', 'n', '.', 'k', '.', '.', '.', 'r']

['p', 'p', 'p', 'b', 'n', 'p', 'p', '.']

['.', '.', '.', '.', '.', '.', '.', '.']

['.', '.', '.', 'q', 'P', '.', '.', 'p']

['.', '.', 'P', 'N', '.', '.', '.', '.']

['.', '.', 'R', '.', '.', 'P', '.', '.']

['P', 'P', '.', '.', 'B', '.', 'P', 'P']

['.', '.', '.', 'Q', '.', 'R', 'K', '.']

Change at c3 | . -> R
Change at c1 | R -> .
c1c3
